In [ ]:
import os
import torch
from torchvision.io import read_image
from torch.utils import data
from torchvision.transforms.functional import convert_image_dtype
import cv2
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image

In [ ]:
class CityScapeDataSet_val(data.Dataset):
  def __init__(self, dataset_directory = None, img_directory = None, labels_directory = None, name_list = None, image_dimensions = (1024, 512)):
    self.name_list = name_list
    self.dataset_directory = dataset_directory
    self.file_name_list =  [file_name.strip() for file_name in open(os.path.join(dataset_directory, name_list))]
    self.img_directory = img_directory
    self.labels_directory = labels_directory
    self.image_dimensions = image_dimensions
    self.label_map = {7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
                      19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
                      26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18}


  def __len__(self):
    return len(self.file_name_list)

  def __getitem__(self, position):
    img_path = os.path.join(self.img_directory, self.file_name_list[position])
    label_path = os.path.join(self.labels_directory, self.file_name_list[position].replace("_leftImg8bit", "_gtFine_labelIds"))

    #we were reading image directly into tensor, which is wrong. We should read it as array and possibly the DataLoader function will auto convert it into tensor
    #professor suggested using pillow for this part

    #here we read the image and convert it to RGB from BGR
    image_RGB = np.asarray(Image.open(img_path).convert("RGB").resize(self.image_dimensions,  Image.BICUBIC), np.float32)
    image_label = np.asarray(Image.open(label_path).resize(self.image_dimensions,  Image.NEAREST), np.float32)

    size = image_RGB.shape
    image_RGB = image_RGB[:, :, ::-1]  # change to BGR
    image_RGB -= (128,128,128)
    image_RGB = image_RGB.transpose((2, 0, 1))

    #remap label pixels
    
    label_RGB_copy = 255 * np.ones((self.image_dimensions[1], self.image_dimensions[0]), dtype=np.float32)
    for id, value in self.label_map.items():
      label_RGB_copy[image_label == id] = value

    return image_RGB.copy(), label_RGB_copy.copy()